# User Interface
The user interface is expected to be a must-have for the framework. The original framework did not
come with a user interface and even when was added later it was more focused on the test running
aspect of the framework. 

The intention here is to put both requirements and tests at the forefront. Requirements being
first class enables an experience where tests can be more tightly connected to requirements.


In [1]:
#export
from flask import render_template, render_template_string, Blueprint
import re
from functools import reduce

import nbloader
nbloader.install_loader()

from nbdev import patch

utils = __import__('Appendix - B - Utilities')

app = Blueprint('app', __name__)

In [2]:
# Setup the jupyter specifics
from IPython.display import display, HTML, IFrame

jupyter_flask = utils.JupyterFlask(name='automationv2', port=8846, 
                                   static_url_path="/static", static_folder='public')

app = utils.JupyterBlueprint(jupyter_flask, app)

jupyter_flask.register_blueprint(app, url_prefix=None)

## Layout
The application will have a 2 column layout

In [3]:
# export

layout_template = '''<html>
    <head>
        <title>My App</title>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
        <script src="/static/js/tailwindcss.js"></script>
        <script>
            tailwind.config = {
                theme: {
                    extend: {
                        colors: {
                            ngblue: "#00269a"
                        }
                    }
                }
            }
        </script>
        <style type="text/tailwindcss">
            @layer utilities {
            .content-auto {
              content-visibility: auto;
            }
          }
        </style>
    </head>
    
    <body>
        <div>
            <div class="h-screen flex flex-col bg-gray-50">
                <div class="bg-ngblue p-2">
                    <img src="/static/img/automation-logo.png" class="h-8"></img>
                </div>
                <div class="flex-grow flex flex-row overflow-hidden justify-center">
                    <!-- lhs -->
                    <div class="flex-shrink-0 w-1/4 p-4">
                        {% block menucontent %} {% endblock %}
                    </div>
                    <!-- center -->
                    <div class="flex-1 flex flex-col bg-white">
                        <main class="flex-1 overflow-y-auto p-4">
                            <div class="relative">
                                {% block content %} {% endblock %}
                            </div>
                    </div>
                    </main>
                </div>
                <!-- footer -->
                <div class="p-2 text-center text-xs text-gray-500">
                    Copyright 2023
                </div>
            </div>
        </div>
    
        <script src="/static/js/htmx/htmx.min.js" defer></script>
        <script src="/static/js/_hyperscript/_hyperscript.min.js" defer></script>
    </body>
    
</html>
'''
with open('templates/layout.html', 'w') as f:
    f.write(layout_template)

In [4]:
sample_template = '''
{% extends "layout.html" %}

{% block menucontent %}
<ul>
    <li>Link 1</li>
    <li>Link 2</li>
</ul>
{% endblock %}

{% block content %}
<h1>MAIN CONTENT</h1>
{% endblock %}
'''

@app.route('/sample')
def sample():
    return render_template_string(sample_template)


In [5]:
display(IFrame("http://localhost:8846/sample", width=800, height=400))

In [14]:
@app.route('/test1')
def test1():
    return b'''<html>
    <head>
    <script src="http://localhost:8846/static/js/htmx/htmx.min.js" type="application/javascript"></script>
    <script src="http://localhost:8846/static/js/tailwindcss.js" type="application/javascript"></script>
    </head>
    <body>
        <div hx-get="/test2">THIS IS COOL AGAIN</div>
    </body>
    </html>'''

@app.route('/test2')
def test2():
    return b'YES PHILLIP, THIS IS COOL!!'

In [15]:
#import http.client

#conn = http.client.HTTPConnection("localhost:8846")
#conn.request("GET", "/static2/js/htmx/htmx.min.js")
#conn.request("GET", "/test1")
#res = conn.getresponse()

#html = res.read().decode('utf-8')
display(IFrame("http://localhost:8846/test1", width=800, height=400))
#print(html)